In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/25 00:09:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2021-01.csv.gz

/bin/sh: 1: curl: not found


In [3]:
!wc -l ./data/fhvhv_tripdata_2021-01.csv

12 ./data/fhvhv_tripdata_2021-01.csv


In [3]:
%ls data/

fhvhv_tripdata_2021-01.csv.gz  raw/
fhvhv_tripdata_2021-06.csv.gz  spark-defaults.conf/


In [4]:
df = spark.read \
    .option("header", "true") \
    .csv('./data/fhvhv_tripdata_2021-01.csv.gz')

In [5]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [8]:
!head -n 1001 ./data/fhvhv_tripdata_2021-01.csv.gz > ./data/head.csv

In [13]:
!gzip -cd ./data/fhvhv_tripdata_2021-01.csv.gz | head -n 1001 > ./data/head.csv


gzip: stdout: Broken pipe


In [6]:
import pandas as pd

In [14]:
df_pandas = pd.read_csv('./data/head.csv', encoding='utf-8')

In [15]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [16]:
spark.createDataFrame(df_pandas).schema

/opt/bitnami/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/opt/bitnami/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

Integer - 4 bytes
Long - 8 bytes

In [17]:
from pyspark.sql import types

In [18]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [19]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('./data/fhvhv_tripdata_2021-01.csv.gz')

In [20]:
df = df.repartition(24)

In [21]:
df.write.parquet('./data/fhvhv/2021/01/')

In [22]:
df = spark.read.parquet('./data/fhvhv/2021/01/')

In [23]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



SELECT * FROM df WHERE hvfhs_license_num =  HV0003

In [24]:
from pyspark.sql import functions as F

In [25]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2021-01-10 17:40:01|2021-01-10 17:48:10|          97|          25|   null|
|           HV0005|              B02510|2021-01-08 18:19:44|2021-01-08 18:55:57|         138|         265|   null|
|           HV0003|              B02876|2021-01-01 16:47:20|2021-01-01 16:58:28|          50|         163|   null|
|           HV0005|              B02510|2021-01-15 17:50:08|2021-01-15 18:07:24|         163|          79|   null|
|           HV0005|              B02510|2021-01-12 17:26:40|2021-01-12 17:57:57|          47|          74|   null|
|           HV0003|              B02765|2021-01-05 02:00:14|2021-01-05 02:19:39|

In [26]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [27]:
crazy_stuff('B02884')

's/b44'

In [28]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [29]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/9ce| 2021-01-10|  2021-01-10|          97|          25|
|  e/9ce| 2021-01-08|  2021-01-08|         138|         265|
|  e/b3c| 2021-01-01|  2021-01-01|          50|         163|
|  e/9ce| 2021-01-15|  2021-01-15|         163|          79|
|  e/9ce| 2021-01-12|  2021-01-12|          47|          74|
|  s/acd| 2021-01-05|  2021-01-05|          48|          95|
|  e/b38| 2021-01-02|  2021-01-02|          63|          77|
|  e/9ce| 2021-01-06|  2021-01-06|         238|          41|
|  e/acc| 2021-01-02|  2021-01-02|          63|         244|
|  e/acc| 2021-01-24|  2021-01-24|         210|         165|
|  e/b35| 2021-01-16|  2021-01-16|         113|         143|
|  e/9ce| 2021-01-28|  2021-01-28|          91|          89|
|  e/acc| 2021-01-01|  2021-01-01|         231|         148|
|  e/b33| 2021-01-19|  2

In [30]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003')


DataFrame[pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int]

In [32]:
!head -n 10 ./data/head.csv